# Credit Card Fraud Prediction using Local Outlier Factor and Isolation Forest

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn import metrics, tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

RANDOM_SEED = 101
LABELS = ['Normal', 'Fraud']

In [2]:
# Import the Dataset
data = pd.read_csv('Sample-Dataset/CreditCard.csv', sep = ',')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Data Preprocessing

In [3]:
# Taking 10% data only for building the model due to time constraints. Set the percentage in parameter 'frac'
data1 = data.sample(frac = 1, random_state = 42)
data1.shape

(69895, 31)

In [4]:
fraud = data1[data1['Class'] == 1]
valid = data1[data1['Class'] == 0]

In [5]:
print(fraud.shape)
print(valid.shape)

(135, 31)
(69760, 31)


In [6]:
outlier_fraction = len(fraud)/float(len(valid))
outlier_fraction

0.0019352064220183487

# Train Test Split

In [7]:
# Split the data into target and feature variables
columns = data1.columns.tolist()
columns = [c for c in columns if c not in ['Class']]
target  = 'Class'
state = np.random.RandomState(101)
x = data1[columns]
y = data1[target]
x_outlier = state.uniform(low=0, high=1, size=(x.shape[0], x.shape[1]))
print(x.shape)
print(y.shape)

(69895, 30)
(69895,)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size  = 0.2, random_state = 42)

# +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+

# Local Outlier Factor

# &

# Isolation Forest

In [9]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [10]:
Classification = {'Isolation Forest' : IsolationForest(n_estimators = 100, 
                                                       max_samples = len(x),
                                                       contamination = outlier_fraction),
                 
                  'Local Outlier Factor' : LocalOutlierFactor(n_neighbors = 20, 
                                                              contamination = outlier_fraction),
                 
                  'Logistic Regression' : LogisticRegression()}

In [11]:
no_of_outlier = len(fraud)

for i, (clf_name, clf) in enumerate(Classification.items()):
    if clf_name == 'Local Outlier Factor':
        y_pred = clf.fit_predict(x)
        score_prediction = clf.negative_outlier_factor_
    elif clf_name == 'Logistic Regression':
        clf.fit(x, y)
        y_pred = clf.predict(x)
    else:
        clf.fit(x)
        score_prediction = clf.decision_function(x)
        y_pred = clf.predict(x)
        
    y_pred[y_pred == 1]  = 0
    y_pred[y_pred == -1] = 1
    n_error = (y_pred != y).sum()
    
    print("{} : {}".format(clf_name, n_error))
    print("Accuracy Score :", accuracy_score(y, y_pred) )
    print("Classification Report :\n", classification_report(y, y_pred), '\n\n')

Isolation Forest : 137
Accuracy Score : 0.9980399170183847
Classification Report :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     69760
           1       0.49      0.50      0.49       135

    accuracy                           1.00     69895
   macro avg       0.75      0.75      0.75     69895
weighted avg       1.00      1.00      1.00     69895
 


Local Outlier Factor : 251
Accuracy Score : 0.99640889906288
Classification Report :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     69760
           1       0.07      0.07      0.07       135

    accuracy                           1.00     69895
   macro avg       0.54      0.54      0.54     69895
weighted avg       1.00      1.00      1.00     69895
 


Logistic Regression : 135
Accuracy Score : 0.9980685313684813
Classification Report :
               precision    recall  f1-score   support

           0       1.00     